In [ ]:
from QLBM import QLBM, get_velocity_set
import numpy as np
import matplotlib.pyplot as plt
import qiskit_aer
from qiskit import transpile

In [ ]:
# Domain parameters
L = 64               # Domain length
u_0 = 0.1            # Initial velocity field
n = 2                # Modes of the velocity field

# Initial conditions
Psi_0 = 1            # Initial scalar field concentration
Psi_1 = 0.1          # Scalar field disturbance

# Viscosity and time-related parameters
nu = 0.001           # Kinematic viscosity
T = 1 / (nu * 2)     # Total simulation time
k = (2 * np.pi * n) / L  # Wave number based on domain length and mode
delta_t = 1          # Time step size
tau = 1              # Relaxation time

# Diffusion and velocity settings
c_s = 1 / np.sqrt(3)   # Speed of sound in lattice units
D = c_s**2 * (tau - delta_t / 2)  # Diffusion coefficient

# Simulation parameters
NUMBER_DISCRETE_VELOCITIES = 3
wave_length = L / n
timesteps = int(np.ceil(T))       # Number of time steps
N_pron = np.arange(1, 8)          # Range for pronumerations
delta_x_range = L / 2**N_pron     # Range of delta x values
t = np.arange(1, timesteps + 1)   # Time array

# Error arrays
L1_error = np.zeros(len(delta_x_range))
L2_error = np.zeros(len(delta_x_range))
L_inf_error = np.zeros(len(delta_x_range))

In [ ]:
simulator = qiskit_aer.backends.statevector_simulator.StatevectorSimulator()

In [ ]:
# Loop over different delta_x values
for ii, delta_x in enumerate(delta_x_range):

    # Set up LBM domain and initial conditions
    L_lbm = L / delta_x
    x_LBM = np.arange(0, L_lbm) + 0.5
    Psi_qlbm = np.zeros((timesteps + 1, x_LBM.size))
    Psi_qlbm[0, :] = Psi_0 + Psi_1 * np.cos(k * x_LBM * delta_x)
    C_u = delta_x / delta_t

    # Quantum LBM simulation loop
    for jj in range(timesteps):
        u_LBM = (u_0 * np.cos(nu * jj) / C_u) * np.ones(int(L_lbm))
        qc = QLBM(density_field=Psi_qlbm[jj, :], velocity_field=u_LBM, number_velocities=NUMBER_DISCRETE_VELOCITIES)
        
        # Compile and run the quantum circuit
        compiled_circuit = transpile(qc, simulator)
        result = simulator.run(compiled_circuit).result()
        
        # Process the statevector
        statevector = np.array(result.get_statevector())
        real_part_statevector = np.real(statevector[:len(x_LBM)])
        Psi_qlbm[jj + 1, :] = real_part_statevector * np.linalg.norm(Psi_qlbm[jj, :]) * 2

    # Solve the analytical solution
    x = x_LBM * delta_x
    N_X = len(x_LBM)
    Psi_analytical = np.zeros((timesteps, N_X))
    for jj in range(timesteps):
        cos_term = np.cos(k * x) * np.cos(u_0 * (k / nu) * np.sin(nu * t[jj]))
        sin_term = np.sin(k * x) * np.sin(u_0 * (k / nu) * np.sin(nu * t[jj]))
        Psi_analytical[jj, :] = Psi_0 + Psi_1 * np.exp(-k**2 * D * t[jj]) * (cos_term + sin_term)

    # Delete the initial condition from Psi_qlbm
    Psi_qlbm = np.delete(Psi_qlbm, 0, axis=0)

    # Compute error norms
    L1_error[ii] = np.sum(np.abs(Psi_analytical - Psi_qlbm)) / (N_X * timesteps)
    L2_error[ii] = np.sum((Psi_analytical - Psi_qlbm)**2) / (N_X * timesteps)
    L_inf_error[ii] = np.max(np.abs(Psi_analytical - Psi_qlbm))

In [ ]:
plt.rcParams.update({'font.size': 22})
plt.rcParams['text.usetex'] = True
plt.figure(1,figsize=(10, 6), dpi=300)

plt.plot(L/(delta_x_range),L1_error,label='L1')
plt.xlabel('Number of lattice sites')
plt.ylabel('$\epsilon_{L_1}$')
plt.xscale('log',base=2)
plt.yscale('log')
plt.xlim([L/(delta_x_range[0]),L/(delta_x_range[-1])])
plt.grid(True, which="both", ls="-")
# plt.savefig('L1.pdf', bbox_inches="tight")

In [ ]:
plt.rcParams.update({'font.size': 22})
plt.rcParams['text.usetex'] = True
plt.figure(2,figsize=(10, 6), dpi=300)
plt.plot(L/(delta_x_range),np.sqrt(L2_error),label='L1')
plt.xlabel('Number of lattice sites')
plt.ylabel('$\epsilon_{L_2}$')
plt.xscale('log',base=2)
plt.yscale('log')
plt.xlim([L/(delta_x_range[0]),L/(delta_x_range[-1])])
plt.grid(True, which="both", ls="-")
#plt.savefig('L2.pdf', bbox_inches="tight")

In [ ]:
plt.rcParams.update({'font.size': 22})
plt.rcParams['text.usetex'] = True
plt.figure(3,figsize=(10, 6), dpi=300)
plt.plot(L/(delta_x_range),L_inf_error,label='$L_{\infty}$')
plt.xlabel('Number of lattice sites')
plt.ylabel('$\epsilon_{L_\infty}$')
plt.xscale('log',base=2)
plt.yscale('log')
plt.xlim([L/(delta_x_range[0]),L/(delta_x_range[-1])])
plt.grid(True, which="both", ls="-")
#plt.savefig('Linfty.pdf', bbox_inches="tight")